In [ ]:
#GENERALIZZAZIONE

import numpy as np
import os
import torch
from PIL import Image
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from tensorflow import keras #to load the dataset
from torchvision import datasets, models, transforms
from torchsummary import summary #network summary
from torchvision.models import resnet50, ResNet50_Weights
import glob
from tqdm import tqdm
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 93312000000000

In [ ]:
LABELS = ['malware']

input_path = "/Users/mario/OneDrive/Desktop/Magistrale 2° Anno - 1° Semestre/Progetto AI4C/prova1/"
BATCH_SIZE = 32

data_transforms = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor()
    ])

test_ds = datasets.ImageFolder(input_path, data_transforms)

dataloaders = {
    'test': torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)}

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 1), 
               nn.Sigmoid()).to(device)
               
#We load the model saved before
model.load_state_dict(torch.load('/Users/mario/OneDrive/Desktop/resnet50.h5', map_location=torch.device('cpu')), strict= False)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
#Here we evaluate the ability to generalize of the Image Based network
count = 0
running_loss = 0.0
running_corrects = 0

model.eval()

for inputs, labels in dataloaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device).float()
    labels = torch.unsqueeze(labels, -1)
    outputs = model(inputs).float()
    loss = criterion(outputs, labels)

    preds = torch.round(outputs)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    count+=1

epoch_loss = running_loss / (count*BATCH_SIZE)
epoch_acc = running_corrects.float() / (count*BATCH_SIZE)

print('loss: {:.4f}, acc: {:.4f}'.format(epoch_loss, epoch_acc)) 

loss: 0.4731, acc: 0.7188
